#Polymorphic types for computational linguistic semantics

The lambda notebook supports polymorphic types (variables over types) in certain cases.  Specifically, what is implemented is a version of the Hindley-Milner type system

Mechanically, you can write type variables using `X`, `Y`, and `Z` followed by any number or number of primes.  So for example, `<X,Y15>` is a function from some type `X` to some type `Y15`.  You also may occasionally see some variable types with the letter `I`; these are internal types and you cannot directly use such types.  See below for ways of dealing with them.

In practice, much of formal semantics does not require variable types.  The main use in the lambda notebook is writing general-purpose combinators.  For example, here is a combinator for doing generalized _Predicate Modification_ over arbitrary property types:

In [38]:
%%lamb
gpm = L f_<X,t> : L g_<X,t> : L x_X : f(x) & g(x)

ERROR (parsing): Parsing of assignment to 'gpm' failed with exception:
ERROR (parsing): 


In [28]:
gpm

(λ f_<X,t>: (λ g_<X,t>: (λ x_X: (f_<X,t>(x_X) & g_<X,t>(x_X)))))

The type variable here signals that the function gpm is underspecified for some type `X`.  All of the `X`s in this formula must be consistent, so whatever `X` is, `g` and `f` must have the same type.  We can see this by combining `gpm` with an argument:

In [29]:
gpm(te("L x_e : Cat_<e,t>(x)")).reduce_all()

(λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))

In [30]:
gpm(te("L x_e : Cat_<e,t>(x)")).reduce_all().derivation

What has happened here?  The lambda notebook metalanguage inferred at the initial combination stage (reduction is not required) that `X` must be equal to `e`, a non-variable type, and systematically replaced the type variable across all instances in the function.

A somewhat more sophisticated example is the implementation of type-checking for function-argument combination in the metalanguage.  The most straightforward algorithm you might imagine would go something like:

 1. Is `fun.type` a functional type?  If not, fail.
 2. Does `fun.type.left` equal `arg.type`?  If not, fail.
 3. Otherwise, succeed.
 
This is a perfectly fine procedure for simple types.  For variable types, we can do something a bit more interesting.

 1. Does `fun.type` match `<arg.type,X>`, where `X` is new?  If not, fail.
 2. Otherwise, succeed.
 
The procedure for checking if two types match in Hindely-Milner is called _type unification_, and is an instance of first-order unification.  The algorithm for doing type unification is sometimes called _Algorithm U_, after Damas and Milner (1982), _Principal type-schemes for functional programs_, proceedings of ACM POPL.  For example, the type checking for a typical combination of `<e,t>` with `e` boils down to the following call to the type unification procedure (recall that `tp` invokes the type parser), which finds a variable assignment of `X` to `t` that can unify the two types:

In [5]:
types.poly_system.unify(tp("<e,t>"), tp("<e,X>"))

<e,t>

It is possible to query the variable assignment that unification finds, though most of the time you wouldn't need to do this.  Here is the above example, as well as a somewhat more complicated example of type unification.

In [6]:
types.poly_system.unify_details(tp("<e,t>"), tp("<e,X>"))

In [7]:
types.poly_system.unify_details(tp("<X,<Y,Z>>"), tp("<Z,<X',e>>"))

The algorithm for doing type inference in the metalanguage is an implementation of Damas and Milner's _Algorithm W_.  This amounts to unifying the constraints imposed by function-argument combination across an entire formula, rather than just a single type.  For example, in the generalized PM example above, we can infer both that `X` maps to `e` and that this is consistent across the formula.  In practice, most cases that are relevant to formal semantics are about this simple.

Another example that is useful to consider is the basic function-argument combination of two variable types.  The function's variable gets mapped into a functional type over variables, where the first one matches the argument's variable type.

In [8]:
%te f_Z(a_Y)

f_<Y,X>(a_Y)

Further inference is of course possible in context, for example, binary truth-functional operators would force `X` in the above result to get mapped to `t`.

In [9]:
%te f_X(a_Y) & p_t

(f_<Y,t>(a_Y) & p_t)

### Let scoping

If you were to run pure type unification from the `types` module on `Z` and `Y` you would get a very different result.

In [5]:
types.poly_system.unify_fa(tp("Z"), tp("Y"))

(<Y,I27>, Y, I27)

Here, the `I`-prefixed type variable is a new (sometimes termed _fresh_) type variable (the exact number may vary substantially).  For human readability, the output of the `%te` magic reduces the type variables it uses (via a version of alpha conversion) to have low numbers, and not be internal types.  Why is this legitimate?  The result of `%te` as well as assignment in the `%lamb` magic have their type variables in the scope of a `let` operator.  A `let` operator binds type variables and treats them as _generic_.  A variable inside a `let` operator does not have to obey constraints imposed on a variable of the same name not in the scope of that `let`.  Let-scoped expressions may therefore safely have their variables renamed as long as internal type constraints are obeyed.

For purposes of formal semantics, one does not typically have to pay much attention to let-scoping or mess with it (in direct contrast to functional programming languages), and so the metalanguage does not indicate it or allow for much manipulation of it.  It is possible to disable let in direct calls to `te`, but currently I don't see that this should be much needed.  Here is the above function-argument example without a let operator, and then with expicit variable renaming performed.  Note that here the input `X` bears no relation to the output `X`.

In [6]:
te("f_X(a_Y)", let=False)

f_<Y,I28>(a_Y)

In [7]:
te("f_X(a_Y)", let=False).compact_type_vars()

f_<Y,X>(a_Y)

In programming languages (/programming language theory) a typical syntax for the let operator is something like:

    let f = lambda x: x
    in
        f(y)
    end

The type of `x` in this sort of syntax would be treated as generic relative to any type constraints imposed or inferred in the body.  This syntax is implicit in the `%%lamb` magic: any assignment to a variable or lexical item is treated as in the definitional part of a `let` statement, and any use later of that variable/item is treated as having generic type variables.

Here is another example, the so-called _Geach combinator_.

In [1]:
%%lamb
geach = L g_<Y,Z> : L f_<X,Y> : L x_X : g(f(x))

${geach}_{\langle{}\langle{}Y,Z\rangle{},\langle{}\langle{}X,Y\rangle{},\langle{}X,Z\rangle{}\rangle{}\rangle{}}\:=\:\lambda{} g_{\langle{}Y,Z\rangle{}} \: . \: \lambda{} f_{\langle{}X,Y\rangle{}} \: . \: \lambda{} x_{X} \: . \: {g}_{\langle{}Y,Z\rangle{}}({f}_{\langle{}X,Y\rangle{}}({x}_{X}))$

This is a combinator version of function composition, for functions of arbitrary types.  `gc(g)(f)(x)` is equivalent to `(g * f)(x)` where `*` is function composition.  Since the metalanguage allows for `*` to signal function composition, we can see this directly:

In [2]:
%%lamb
f = L x_n : x+1
g = L x_n : x>3

${f}_{\langle{}n,n\rangle{}}\:=\:\lambda{} x_{n} \: . \: ({x}_{n} + {1}_{n})$<br />
${g}_{\langle{}n,t\rangle{}}\:=\:\lambda{} x_{n} \: . \: ({x}_{n} > {3}_{n})$

In [3]:
g * f

(λ x_n: ((x_n + 1_n) > 3_n))

In [4]:
geach(g)(f).reduce_all()

(λ x_n: ((x_n + 1_n) > 3_n))

In [5]:
%lamb h = L p_t : p & Q_t

${h}_{\langle{}t,t\rangle{}}\:=\:\lambda{} p_{t} \: . \: ({p}_{t} \wedge{} {Q}_{t})$

In [6]:
h * g

(λ x_n: ((x_n > 3_n) & Q_t))

In [7]:
geach(h)(g).reduce_all().derivation.trace()

In [8]:
geach(h)(g).derivation #[0].derivation[0].origin[0]

In [9]:
geach(h).derivation

For the sake of convenience, there is a special variable notated and entered as `?`.  Simplifying slightly, instances of `?` types do not equal each other (i.e. it is a shorthand for using only fresh type variables).  You can use this when you simply don't know the type, for example, and the `?`s will get translated into regular type variables.  (Note that if you use this without `let`, you may get slightly counterintuitive results unless the type variables are renamed.)

In [3]:
%te f_?(y_?)

f_<X,X'>(y_X)